# Проект 3. О вкусной и здоровой пище 

построение модели, которая предсказывает рейтинг ресторана

In [1]:
import sys
import numpy as np # для работы с массивами
import pandas as pd # для работы DataFrame 
import seaborn as sns # библиотека для визуализации статистических данных
import matplotlib.pyplot as plt # для построения графиков
from collections import Counter
import ast
import re
%matplotlib inline

In [2]:
df = pd.read_csv('D:/main_task.csv')

In [3]:
df.describe()

,Ranking,Rating,Number of Reviews
count,40000.000000,40000.000000,37457.000000
mean,3676.028525,3.993037,124.825480
std,3708.749567,0.668417,295.666352
min,1.000000,1.000000,2.000000
25%,973.000000,3.500000,9.000000
50%,2285.000000,4.000000,33.000000
75%,5260.000000,4.500000,115.000000
max,16444.000000,5.000000,9660.000000


In [4]:
df

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963
...,...,...,...,...,...,...,...,...,...,...
39995,id_499,Milan,"['Italian', 'Vegetarian Friendly', 'Vegan Opti...",500.0,4.5,$$ - $$$,79.0,"[['The real Italian experience!', 'Wonderful f...",/Restaurant_Review-g187849-d2104414-Reviews-Ro...,d2104414
39996,id_6340,Paris,"['French', 'American', 'Bar', 'European', 'Veg...",6341.0,3.5,$$ - $$$,542.0,"[['Parisian atmosphere', 'Bit pricey but inter...",/Restaurant_Review-g187147-d1800036-Reviews-La...,d1800036
39997,id_1649,Stockholm,"['Japanese', 'Sushi']",1652.0,4.5,NaN,4.0,"[['Good by swedish standards', 'A hidden jewel...",/Restaurant_Review-g189852-d947615-Reviews-Sus...,d947615
39998,id_640,Warsaw,"['Polish', 'European', 'Eastern European', 'Ce...",641.0,4.0,$$ - $$$,70.0,"[['Underground restaurant', 'Oldest Restaurant...",/Restaurant_Review-g274856-d1100838-Reviews-Ho...,d1100838


Первоначальная версия датасета состоит из десяти столбцов, содержащих следующую информацию:

* Restaurant_id — идентификационный номер ресторана / сети ресторанов;
* City — город, в котором находится ресторан;
* Cuisine Style — кухня или кухни, к которым можно отнести блюда, предлагаемые в ресторане;
* Ranking — место, которое занимает данный ресторан среди всех ресторанов своего города;
* Rating — рейтинг ресторана по данным TripAdvisor (именно это значение должна будет предсказывать модель);
* Price Range — диапазон цен в ресторане;
* Number of Reviews — количество отзывов о ресторане;
* Reviews — данные о двух отзывах, которые отображаются на сайте ресторана;
* URL_TA — URL страницы ресторана на TripAdvisor;
* ID_TA — идентификатор ресторана в базе данных TripAdvisor.

### Анализ датасета

Есть незаполненные значения в полях Cuisine Style，Price Range и Number of Reviews

In [5]:
print(f'Незаполненные значения:\n{df.isnull().sum()}')

Незаполненные значения:
Restaurant_id            0
City                     0
Cuisine Style         9283
Ranking                  0
Rating                   0
Price Range          13886
Number of Reviews     2543
Reviews                  0
URL_TA                   0
ID_TA                    0
dtype: int64


Посмотрим на типы данных в столбцах

In [6]:
for c in df.columns:
    print(f'{c} : {df[c].dtype}')

Restaurant_id : object
City : object
Cuisine Style : object
Ranking : float64
Rating : float64
Price Range : object
Number of Reviews : float64
Reviews : object
URL_TA : object
ID_TA : object


In [7]:
for c in df.columns:
    print(f'{c} : {type(df[c][0])}')

Restaurant_id : <class 'str'>
City : <class 'str'>
Cuisine Style : <class 'str'>
Ranking : <class 'numpy.float64'>
Rating : <class 'numpy.float64'>
Price Range : <class 'str'>
Number of Reviews : <class 'numpy.float64'>
Reviews : <class 'str'>
URL_TA : <class 'str'>
ID_TA : <class 'str'>


Удалим столбцы, содержащие данные типа object, и заполним пропущенные значения (None или NaN) средним арифметическим для  всего столбца.

In [8]:
df_train = df.copy()
for c in df_train.columns:
    if df_train[c].dtype == "object":
        df_train = df_train.drop(c, axis = 1)
    
for c in df_train.columns:
    m = df_train[c].mean()
    df_train[c] = df_train[c].fillna(m)
    
df_train

,Ranking,Rating,Number of Reviews
0,5570.0,3.5,194.0
1,1537.0,4.0,10.0
2,353.0,4.5,688.0
3,3458.0,5.0,3.0
4,621.0,4.0,84.0
...,...,...,...
39995,500.0,4.5,79.0
39996,6341.0,3.5,542.0
39997,1652.0,4.5,4.0
39998,641.0,4.0,70.0


## Модель из исходных данных

построим модель из исходных данных и получим метрику (Mean Absolute Error), показывающую абсолютное значение отклонения предсказанных значений от фактических:

In [9]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)  
X = df_train.drop(['Rating'], axis = 1)  
y = df_train['Rating']  
  
# Загружаем специальный инструмент для разбивки:  
from sklearn.model_selection import train_test_split  
  
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
# Для тестирования мы будем использовать 25% от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)  

In [10]:
# Импортируем необходимые библиотеки:  
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели  
  
# Создаём модель  
regr = RandomForestRegressor(n_estimators=100)  
  
# Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train)  
  
# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test)  

Для оценки точности прогнозов, сделанных моделью, мы будем использовать метрику (показатель), которая называется Mean Absolute Error, и представляет собой среднее абсолютное значение отклонения предсказанных значений от фактических:

In [11]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются  
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.  
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))  

MAE: 0.43006141805555553


## Ответим на вопросы

### Вопросы о ценах

* Сколько вариантов непустых значений встречается в столбце Price Range?
* Как в датафрейме обозначается самый низкий уровень цен?
* Как в датафрейме обозначается самый высокий уровень цен?

In [12]:
df['Price Range'].value_counts()

$$ - $$$    18412
$            6279
$$$$         1423
Name: Price Range, dtype: int64

In [13]:
price_set = set(df['Price Range'].values)
price_set

{'$', '$$ - $$$', '$$$$', nan}

Сколько ресторанов относятся к среднему ценовому сегменту?

In [14]:
df[df['Price Range'] == '$$ - $$$']['Restaurant_id'].count()

18412

### Вопрос о городах

Сколько городов представлено в наборе данных?

In [15]:
df['City'].value_counts().count()

31

### Вопросы о кухнях

Сколько типов кухонь представлено в наборе данных?

In [16]:
df_cousine = df.copy()
def t(c):
    if isinstance(c, str):  
        return ast.literal_eval(c)
    return []

df_cousine['Cuisine Style'] = df_cousine['Cuisine Style'].apply(t)
print(df_cousine['Cuisine Style'])
cousines_set = set(df_cousine['Cuisine Style'].sum())

0                        [European, French, International]
1                                                       []
2        [Japanese, Sushi, Asian, Grill, Vegetarian Fri...
3                                                       []
4          [German, Central European, Vegetarian Friendly]
                               ...                        
39995    [Italian, Vegetarian Friendly, Vegan Options, ...
39996    [French, American, Bar, European, Vegetarian F...
39997                                    [Japanese, Sushi]
39998    [Polish, European, Eastern European, Central E...
39999                                            [Spanish]
Name: Cuisine Style, Length: 40000, dtype: object


In [17]:
cousines_set

{'Afghani',
 'African',
 'Albanian',
 'American',
 'Arabic',
 'Argentinean',
 'Armenian',
 'Asian',
 'Australian',
 'Austrian',
 'Azerbaijani',
 'Balti',
 'Bangladeshi',
 'Bar',
 'Barbecue',
 'Belgian',
 'Brazilian',
 'Brew Pub',
 'British',
 'Burmese',
 'Cafe',
 'Cajun & Creole',
 'Cambodian',
 'Canadian',
 'Caribbean',
 'Caucasian',
 'Central American',
 'Central Asian',
 'Central European',
 'Chilean',
 'Chinese',
 'Colombian',
 'Contemporary',
 'Croatian',
 'Cuban',
 'Czech',
 'Danish',
 'Delicatessen',
 'Diner',
 'Dutch',
 'Eastern European',
 'Ecuadorean',
 'Egyptian',
 'Ethiopian',
 'European',
 'Fast Food',
 'Filipino',
 'French',
 'Fujian',
 'Fusion',
 'Gastropub',
 'Georgian',
 'German',
 'Gluten Free Options',
 'Greek',
 'Grill',
 'Halal',
 'Hawaiian',
 'Healthy',
 'Hungarian',
 'Indian',
 'Indonesian',
 'International',
 'Irish',
 'Israeli',
 'Italian',
 'Jamaican',
 'Japanese',
 'Korean',
 'Kosher',
 'Latin',
 'Latvian',
 'Lebanese',
 'Malaysian',
 'Mediterranean',
 'Mexic

In [18]:
print(f'Количество кухонь: {len(cousines_set)}')

Количество кухонь: 125


Какая кухня представлена в наибольшем количестве ресторанов?

In [19]:
df_cousine_exp = df_cousine.explode('Cuisine Style')
MP = df_cousine_exp['Cuisine Style'].value_counts().head(1).index[0]
print(f'Самая популярная: {MP}')

Самая популярная: Vegetarian Friendly


Какое среднее количество кухонь предлагается в одном ресторане? Если в данных отсутствует информация о типах кухонь, то считайте, что в этом ресторане предлагается только один тип кухни. Ответ округлите до одного знака после запятой.

In [20]:
def cousine_num(c):
    res = len(c)
    if res == 0:
        return 1
    return res

df_cousine['cuisines'] = df_cousine['Cuisine Style'].apply(cousine_num)

In [21]:
MC = df_cousine['cuisines'].mean()
print(f'Среднее число кухонь: {round(MC, 1)}')

Среднее число кухонь: 2.6


### Вопросы об отзывах

Когда был оставлен самый свежий отзыв?

In [22]:
reviews_list = [] 

def r1(r_id, r_ta, r):
    if isinstance(r, str):
        r = re.sub("(\W)nan(\W)", "\\1'nan'\\2", r)
        l = ast.literal_eval(r)
        comments = l[0]
        dates = l[1]
        for i, r in enumerate(comments):
            reviews_list.append([r_id, r_ta, r, dates[i]])
        return c

df_cousine.apply(lambda x: r1(x['Restaurant_id'], x['ID_TA'], x['Reviews']), axis = 1)
df_reviews = pd.DataFrame(reviews_list, columns = ['Restaurant_id', 'ID_TA', 'Review', 'Date'])

In [23]:
df_reviews['Date'] = pd.to_datetime(df_reviews['Date'])

In [24]:
df_reviews.sort_values(by='Date').tail(1)['Date'].item()

Timestamp('2018-02-26 00:00:00')

Самый свежий отзыв был оставлен 2018-02-26

Какое максимальное количество дней отделяет даты публикации отзывов, размещённых на сайте ресторана? Введите количество дней в виде целого числа.

In [25]:
df_diffs_rid = df_reviews.groupby('Restaurant_id').agg(np.ptp)

In [26]:
df_diffs_rid['Date'] = df_diffs_rid['Date'].dt.days

In [27]:
df_diffs_rid

,Date
Restaurant_id,
id_0,182
id_1,59
id_10,112
id_100,135
id_1000,1357
...,...
id_999,1582
id_9991,151
id_9992,229


In [28]:
df_diffs = df_reviews.groupby('ID_TA').agg(np.ptp)

In [29]:
df_diffs['Date'] = df_diffs['Date'].dt.days

In [30]:
df_diffs.rename(columns={"Date": "Diffs"}).sort_values(by='Diffs').tail(1)['Diffs'].item()

3207

 Максимальное количество дней отделяет даты публикации отзывов, размещённых на сайте ресторана = 3207

## Добавляем новые признаки

In [31]:
df_mod = df_cousine.copy()

### Создадим dummy variables для кухонь

In [32]:
cousines_set

{'Afghani',
 'African',
 'Albanian',
 'American',
 'Arabic',
 'Argentinean',
 'Armenian',
 'Asian',
 'Australian',
 'Austrian',
 'Azerbaijani',
 'Balti',
 'Bangladeshi',
 'Bar',
 'Barbecue',
 'Belgian',
 'Brazilian',
 'Brew Pub',
 'British',
 'Burmese',
 'Cafe',
 'Cajun & Creole',
 'Cambodian',
 'Canadian',
 'Caribbean',
 'Caucasian',
 'Central American',
 'Central Asian',
 'Central European',
 'Chilean',
 'Chinese',
 'Colombian',
 'Contemporary',
 'Croatian',
 'Cuban',
 'Czech',
 'Danish',
 'Delicatessen',
 'Diner',
 'Dutch',
 'Eastern European',
 'Ecuadorean',
 'Egyptian',
 'Ethiopian',
 'European',
 'Fast Food',
 'Filipino',
 'French',
 'Fujian',
 'Fusion',
 'Gastropub',
 'Georgian',
 'German',
 'Gluten Free Options',
 'Greek',
 'Grill',
 'Halal',
 'Hawaiian',
 'Healthy',
 'Hungarian',
 'Indian',
 'Indonesian',
 'International',
 'Irish',
 'Israeli',
 'Italian',
 'Jamaican',
 'Japanese',
 'Korean',
 'Kosher',
 'Latin',
 'Latvian',
 'Lebanese',
 'Malaysian',
 'Mediterranean',
 'Mexic

In [33]:
def find_item(item, items):
    if item in items:
        return 1
    return 0
    
for c in cousines_set:
    df_mod[c] = df_cousine['Cuisine Style'].apply(lambda x: find_item(c,x))

### Заполним пропуски в цене значением None и создадим dummy variables для цен

In [34]:
df_mod['Price Range'] = df_mod['Price Range'].fillna('None')

In [35]:
price_set = set(df_mod['Price Range'].values)
price_set

{'$', '$$ - $$$', '$$$$', 'None'}

In [36]:
 for c in price_set:
    df_mod[c] = df_mod['Price Range'].apply(lambda x: find_item(c,x))

### Создадим dummy variables из городов

In [37]:
city_set = set(df_mod['City'].values)
city_set

{'Amsterdam',
 'Athens',
 'Barcelona',
 'Berlin',
 'Bratislava',
 'Brussels',
 'Budapest',
 'Copenhagen',
 'Dublin',
 'Edinburgh',
 'Geneva',
 'Hamburg',
 'Helsinki',
 'Krakow',
 'Lisbon',
 'Ljubljana',
 'London',
 'Luxembourg',
 'Lyon',
 'Madrid',
 'Milan',
 'Munich',
 'Oporto',
 'Oslo',
 'Paris',
 'Prague',
 'Rome',
 'Stockholm',
 'Vienna',
 'Warsaw',
 'Zurich'}

In [38]:
 for c in city_set:
    df_mod[c] = df_mod['City'].apply(lambda x: find_item(c,x))

### Создадим список городов и самых популярных кухонь в них

In [39]:
city_cuisine_list = []

def f1(c):
    city = c['City'].values[0]
    cuisines = c['Cuisine Style'].dropna().sum()
    cuisine = Counter(cuisines).most_common(1)[0][0]
    city_cuisine_list.append([city, cuisine])

df_mod.groupby('City').apply(f1)

df_city_top_cuisine = pd.DataFrame(city_cuisine_list, columns=['City', 'Cuisine Style'])
df_city_top_cuisine.set_index('City', inplace=True)

In [40]:
df_city_top_cuisine

,Cuisine Style
City,
Amsterdam,European
Athens,Greek
Barcelona,Spanish
Berlin,Vegetarian Friendly
Bratislava,European
Brussels,European
Budapest,European
Copenhagen,European
Dublin,Vegetarian Friendly


### Если в ресторане отсутствует значение кухни, то подставляем самую популярную кухню в городе

In [41]:
def f2(x):
    city = x['City']
    if x['Cuisine Style'] == []:
        x['Cuisine Style'] = [df_city_top_cuisine.loc[city].item()]
    return x

df_mod = df_mod.apply(f2, axis = 1)

In [42]:
df_mod

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,...,Warsaw,Lisbon,Munich,Bratislava,Geneva,Oporto,Paris,Ljubljana,Budapest,Edinburgh
0,id_5569,Paris,"[European, French, International]",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,...,0,0,0,0,0,0,1,0,0,0
1,id_1535,Stockholm,[European],1537.0,4.0,None,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,...,0,0,0,0,0,0,0,0,0,0
2,id_352,London,"[Japanese, Sushi, Asian, Grill, Vegetarian Fri...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,...,0,0,0,0,0,0,0,0,0,0
3,id_3456,Berlin,[Vegetarian Friendly],3458.0,5.0,None,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,...,0,0,0,0,0,0,0,0,0,0
4,id_615,Munich,"[German, Central European, Vegetarian Friendly]",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,id_499,Milan,"[Italian, Vegetarian Friendly, Vegan Options, ...",500.0,4.5,$$ - $$$,79.0,"[['The real Italian experience!', 'Wonderful f...",/Restaurant_Review-g187849-d2104414-Reviews-Ro...,d2104414,...,0,0,0,0,0,0,0,0,0,0
39996,id_6340,Paris,"[French, American, Bar, European, Vegetarian F...",6341.0,3.5,$$ - $$$,542.0,"[['Parisian atmosphere', 'Bit pricey but inter...",/Restaurant_Review-g187147-d1800036-Reviews-La...,d1800036,...,0,0,0,0,0,0,1,0,0,0
39997,id_1649,Stockholm,"[Japanese, Sushi]",1652.0,4.5,None,4.0,"[['Good by swedish standards', 'A hidden jewel...",/Restaurant_Review-g189852-d947615-Reviews-Sus...,d947615,...,0,0,0,0,0,0,0,0,0,0
39998,id_640,Warsaw,"[Polish, European, Eastern European, Central E...",641.0,4.0,$$ - $$$,70.0,"[['Underground restaurant', 'Oldest Restaurant...",/Restaurant_Review-g274856-d1100838-Reviews-Ho...,d1100838,...,1,0,0,0,0,0,0,0,0,0


In [43]:
df_mod2 = df_mod.copy()

### Добавим в датасет время между последними комментариями о ресторане

In [44]:
df_mod2 = df_mod.merge(df_diffs, on='ID_TA', how='outer')

In [45]:
df_mod2 = df_mod2.rename(columns = {'Date' : 'Diffs'})

In [46]:
m = df_mod2['Diffs'].median()
df_mod2['Diffs'] = df_mod2['Diffs'].fillna(m)

### Если количество комментариев отсутствует, посчитаем вручную количество комментариев для этого ID_TA 

In [47]:
def r2(x):
    if np.isnan(x['Number of Reviews']) == True:
        x['Number of Reviews'] = df_reviews[df_reviews['ID_TA'] == x['ID_TA']]['Date'].count()
    return x

df_mod2 = df_mod2.apply(r2, axis = 1)

## Добавим страны

Добавим страны, соответствующие городам и создадим из них dummy variables признаки

In [48]:
city_list = list(city_set)
city_list.sort()
city_list

['Amsterdam',
 'Athens',
 'Barcelona',
 'Berlin',
 'Bratislava',
 'Brussels',
 'Budapest',
 'Copenhagen',
 'Dublin',
 'Edinburgh',
 'Geneva',
 'Hamburg',
 'Helsinki',
 'Krakow',
 'Lisbon',
 'Ljubljana',
 'London',
 'Luxembourg',
 'Lyon',
 'Madrid',
 'Milan',
 'Munich',
 'Oporto',
 'Oslo',
 'Paris',
 'Prague',
 'Rome',
 'Stockholm',
 'Vienna',
 'Warsaw',
 'Zurich']

In [49]:
country_city_set = [
    'Netherlands',
    'Greece',
    'Spain',
    'Germany',
    'Slovakia',
    'Belgium',
    'Hungary',
    'Denmark',
    'Ireland',
    'Scotland',
    'Switzerland',
    'Germany',
    'Finland',
    'Poland',
    'Portugal',
    'Slovenia',
    'England',
    'Luxembourg',
    'French',
    'Spain',
    'Italy',
    'Germany',
    'Portugal',
    'Norway',
    'French',
    'Chech Republic',
    'Italy',
    'Sweden',
    'Austria',
    'Poland',
    'Switzerland'
]

Добавим население городов

In [50]:
city_citizen_set = [
    869709,
    664046,
    1628552,
    3613495,
    431214,
    176545,
    1751010,
    616098,
    544107,
    482005,
    198899,
    1830584,
    639227,
    762776,
    506654,
    281226,
    8538689,
    119215,
    506615,
    3244730,
    1372434,
    1456039,
    214935,
    681067,
    2206488,
    1294513,
    2864466,
    789024,
    1897491,
    1766212,
    415367
]


In [51]:
d = {'City' : city_list, 'Country' : country_city_set, 'Citizen' : city_citizen_set}
df_city_country = pd.DataFrame(d)
df_city_country.set_index('City', inplace=True)

In [52]:
df_city_country

,Country,Citizen
City,,
Amsterdam,Netherlands,869709
Athens,Greece,664046
Barcelona,Spain,1628552
Berlin,Germany,3613495
Bratislava,Slovakia,431214
Brussels,Belgium,176545
Budapest,Hungary,1751010
Copenhagen,Denmark,616098
Dublin,Ireland,544107


Добавим население городов в датасет

In [53]:
df_mod2 = df_mod2.merge(df_city_country, on='City', how='left')

Добавим страны как dummy variables признаки

In [54]:
 for c in set(country_city_set):
    df_mod2[c] = df_mod2['Country'].apply(lambda x: find_item(c,x))

## Добавим разделение на старую и новую Европу
Бельгия, Германия, Италия, Люксембург, Нидерланды, Франция, Великобритания, Дания, Ирландия, Греция, Испания, Португалия, Австрия, Финляндия, Швеция


In [55]:
old_europe_list = [
    'Austria',
    'Belgium',
    'Denmark',
    'England',
    'Finland',
    'French',
    'Germany',
    'Greece',
    'Ireland',
    'Italy',
    'Luxembourg',
    'Netherlands',
    'Portugal',
    'Scotland',
    'Spain',
    'Sweden',
]

Добавим признак того, что город принадлежит к старой Европе

In [56]:
df_mod2['OE'] = df_mod2['Country'].apply(lambda x: 1 if x in old_europe_list else 0)

## Добавим признак о том, что город - это столица

In [57]:
capitals_list = [
 'Amsterdam',
 'Athens',
 'Berlin',
 'Bratislava',
 'Brussels',
 'Budapest',
 'Copenhagen',
 'Dublin',
 'Edinburgh',
 'Helsinki',
 'Lisbon',
 'Ljubljana',
 'London',
 'Luxembourg',
 'Madrid',
 'Oslo',
 'Paris',
 'Prague',
 'Rome',
 'Stockholm',
 'Vienna',
 'Warsaw',
]

In [58]:
df_mod2['CC'] = df_mod2['City'].apply(lambda x: 1 if x in capitals_list else 0)

## Построим модель по датасету с новыми признаками

In [59]:
df_train = df_mod2.copy()

In [60]:
print(f'Незаполненные значения:\n{df_train.isnull().sum().sort_values()}')

Незаполненные значения:
Restaurant_id    0
American         0
Xinjiang         0
Contemporary     0
Brew Pub         0
                ..
Delicatessen     0
Brazilian        0
Spanish          0
Egyptian         0
CC               0
Length: 197, dtype: int64


In [61]:
for c in df_train.columns:
    if df_train[c].dtype == "object":
        df_train = df_train.drop(c, axis = 1)
        
df_train

,Ranking,Rating,Number of Reviews,cuisines,Native American,Chilean,Azerbaijani,Czech,Central American,Bangladeshi,...,Slovenia,Sweden,Netherlands,Ireland,Denmark,Finland,Switzerland,Belgium,OE,CC
0,5570.0,3.5,194.0,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
1,1537.0,4.0,10.0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,1
2,353.0,4.5,688.0,7,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
3,3458.0,5.0,3.0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
4,621.0,4.0,84.0,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,500.0,4.5,79.0,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
39996,6341.0,3.5,542.0,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
39997,1652.0,4.5,4.0,2,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,1
39998,641.0,4.0,70.0,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [62]:
# Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)  
X = df_train.drop(['Rating'], axis = 1)  
y = df_train['Rating']  
  
print(X.shape)
print(y.shape)

# Загружаем специальный инструмент для разбивки:  
from sklearn.model_selection import train_test_split  
  
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
# Для тестирования мы будем использовать 25% от исходного датасета.  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25) 

(40000, 188)
(40000,)


In [63]:
# Импортируем необходимые библиотеки:  
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели  
  
# Создаём модель  
regr = RandomForestRegressor(n_estimators=100)  
  
# Обучаем модель на тестовом наборе данных  
regr.fit(X_train, y_train)  
  
# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.  
# Предсказанные значения записываем в переменную y_pred  
y_pred = regr.predict(X_test)  

In [64]:
y_test.shape

(10000,)

In [65]:
y_pred.shape

(10000,)

In [66]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются  
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.  
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))  

MAE: 0.211024
